Ref: https://www.tensorflow.org/tutorials/estimators/
But modified a bit...

In [5]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tempfile
import urllib

import numpy as np
import tensorflow as tf




In [8]:
train_file = tempfile.NamedTemporaryFile(delete=False)
urllib.urlretrieve("http://download.tensorflow.org/data/abalone_train.csv", train_file.name)
train_file_name = train_file.name
train_file.close()
print("Training data is downloaded to %s" % train_file_name)


Training data is downloaded to /tmp/tmpE4JzQM


In [9]:
test_file = tempfile.NamedTemporaryFile(delete=False)
urllib.urlretrieve("http://download.tensorflow.org/data/abalone_test.csv", test_file.name)
test_file_name = test_file.name
test_file.close()
print("Test data is downloaded to %s" % test_file_name)


Test data is downloaded to /tmp/tmp8CJnbb


In [10]:
predict_file = tempfile.NamedTemporaryFile(delete=False)
urllib.urlretrieve("http://download.tensorflow.org/data/abalone_predict.csv", predict_file.name)
predict_file_name = predict_file.name
predict_file.close()
print("Prediction data is downloaded to %s" % predict_file_name)


Prediction data is downloaded to /tmp/tmpc38b8B


In [13]:
# Training examples
training_set = tf.contrib.learn.datasets.base.load_csv_without_header(
    filename=train_file_name,
    target_dtype=np.int,
    features_dtype=np.float64)

# Test examples
test_set = tf.contrib.learn.datasets.base.load_csv_without_header(
    filename=test_file_name,
    target_dtype=np.int,
    features_dtype=np.float64)

# Set of 7 examples for which to predict abalone ages
prediction_set = tf.contrib.learn.datasets.base.load_csv_without_header(
    filename=predict_file_name,
    target_dtype=np.int,
    features_dtype=np.float64)


In [21]:
def model_fn(features, targets, mode, params):
  """Model function for Estimator."""

  # Connect the first hidden layer to input layer
  # (features) with relu activation
  first_hidden_layer = tf.contrib.layers.relu(features, 10)

  # Connect the second hidden layer to first hidden layer with relu
  second_hidden_layer = tf.contrib.layers.relu(first_hidden_layer, 10)

  # Connect the output layer to second hidden layer (no activation fn)
  output_layer = tf.contrib.layers.linear(second_hidden_layer, 1)

  # Reshape output layer to 1-dim Tensor to return predictions
  predictions = tf.reshape(output_layer, [-1])
  predictions_dict = {"ages": predictions}

  # Calculate loss using mean squared error
  loss = tf.contrib.losses.mean_squared_error(predictions, targets)

  train_op = tf.contrib.layers.optimize_loss(
      loss=loss,
      global_step=tf.contrib.framework.get_global_step(),
      learning_rate=params["learning_rate"],
      optimizer="Adam")

  return predictions_dict, loss, train_op



In [22]:
tf.logging.set_verbosity(tf.logging.INFO)
LEARNING_RATE = 0.001
# Set model params
model_params = {"learning_rate": LEARNING_RATE}

# Build 2 layer fully connected DNN with 10, 10 units respectively.
nn = tf.contrib.learn.Estimator(
  model_fn=model_fn, params=model_params)

# Fit
nn.fit(x=training_set.data, y=training_set.target, steps=5000)

# Score accuracy
ev = nn.evaluate(x=test_set.data, y=test_set.target, steps=1)
loss_score = ev["loss"]
print("Loss: %s" % loss_score)

# Print out predictions
predictions = nn.predict(x=prediction_set.data,
                           as_iterable=True)
for i, p in enumerate(predictions):
    print("Prediction %s: %s" % (i + 1, p["ages"]))


INFO:tensorflow:Using config: {'task': 0, 'save_summary_steps': 100, 'keep_checkpoint_max': 5, '_is_chief': True, 'save_checkpoints_secs': 600, 'evaluation_master': '', 'tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, 'master': '', 'keep_checkpoint_every_n_hours': 10000, '_job_name': None, 'cluster_spec': None, 'tf_random_seed': None, 'num_ps_replicas': 0}
INFO:tensorflow:Setting feature info to TensorSignature(dtype=tf.float64, shape=TensorShape([Dimension(None), Dimension(7)]), is_sparse=False)
INFO:tensorflow:Setting targets info to TensorSignature(dtype=tf.int64, shape=TensorShape([Dimension(None)]), is_sparse=False)
Instructions for updating:
Use mean_squared_error.
INFO:tensorflow:Create CheckpointSaverHook
INFO:tensorflow:loss = 112.797, step = 1
INFO:tensorflow:Saving checkpoints for 1 into /tmp/tmpyTT0Gj/model.ckpt.
INFO:tensorflow:loss = 97.0957, step = 101
INFO:tensorflow:loss = 51.2427, step = 201
INFO:tensorflow:loss = 12.2182, step = 301
INFO:tensorflow

Loss: 5.41782


INFO:tensorflow:Loading model from checkpoint: /tmp/tmpyTT0Gj/model.ckpt-5000-?????-of-00001.


Prediction 1: 4.72838905984
Prediction 2: 11.0697361746
Prediction 3: 7.0667246748
Prediction 4: 11.3843663267
Prediction 5: 11.2302104386
Prediction 6: 9.91030937428
Prediction 7: 11.3227967326
